# Math primer for ML & TensorFlow workshop

This notebook is meant to provide some basic mathematical backgrounds for the Python ML & TensorFlow Workshop.  Three topics are covered:   

1. Structure in Data
2. Basic Concepts in TensorFlow
3. Loss Function and Gradient Descent


## 1. Structure in Data

Intuitively speaking, structure in data means two properties:
* Given some data, one can predict the other data points with some confidence 
* One can compress the data, i.e., store the same amount of information, with less space

For example, consider two arrays:

A = (1, 2, 6, 2, 4, 7)

B = (1, 2, 1, 2, 1, 2)

We might say that A does not have apparent structure, whereas B does.

### Entropy

There are several ways of characterizing structure in data.  For example, if elements in A and B are drawn from probability distributions, then we can define **entropy** as:

$$ H(X)\;=\;- \sum^N_{i=1} p(x_i) \; log_2 p(x_i).$$

To interpret this definition, consider two simple processes: a coin toss and a dice roll.  In each scenario, the outcomes are equally likely, but the numbers of possible outcomes are 2 and 6, respectively.  Therefore (in the unit of bits):

$$ H(coin\;toss)\;=\;- \sum^2_{i=1} {1 \over 2} \; log_2 {1 \over 2} \;=\; 1.0 $$
$$ H(dice\;roll)\;=\;- \sum^6_{i=1} {1 \over 6} \; log_2 {1 \over 6} \;=\; 2.58 $$

So the dice roll process has a higher entropy than coin toss; predicting dice roll outcomes is more difficult, i.e., incurs a larger uncertainty.  Now compare a fair coin toss and a biased coin toss (that, say, returns heads 80% of the time):

$$ H(50/50\;coin\;toss)\;=\;- \sum^2_{i=1} {1 \over 2} \; log_2 {1 \over 2} \;=\; 1.0 $$
$$ H(20/80\;coin\;toss)\;=\;- {1 \over 5} \; log_2 {1 \over 5} - {4 \over 5} \; log_2 {4 \over 5} \;=\; 0.72 $$

So a biased coin toss has lower entropy; predicting its outcome is easier (i.e., less uncertainty) than a fair coin toss.   

<!---
#### Exercise
To continue with the arrays A and B example above, assume these two arrays are representative of the distributions that generated the elements.  Compute the entropy for the distributions that generated A and B. (Note: in real world one almost never knows the actual distribution of a data-generating process.)
--->



### Collinearity

In order to make predictions, we need signals that are information-rich.  For example, imagine the predictive task of identifying defective coins from a large number of coins.  If defective and good coins have exactly the same properties however way one measures them, such as weights, radii, colors, chemical composition, etc., then there is no modeling algorithm that could predict, given such properties, if a coin is defective or good.

Imagine a scenario that defective coins (labeled as positive class **+**) on average weigh less than good ones (labeled as negative class **-**), and that for both classes the weights follow normal distributions. For this feature $x_1$ = weight, we have the following plot.

<img src="./good_signal.png" width="30%" height="30%">

Compare the plot above with the one below, which still have the property that defective coints on average weigh less than good ones.  But in the scenario below it is intuitive to see that the classification task would be more difficult if $x_1$ is the only signal available.  

<img src="./bad_signal.png" width="30%" height="30%">

When there are more than one signals available for a preditive task, it is important to assess if each of the signals carries additional information.  For example, assume besides $x_1$ = weigh, we also have an additional feature $x_2$ = amount of silver in the coin.   

<img src="./correlated_signals.png" width="30%" height="30%">

We can see from the plot above there exists a linear relationship between $x_1$ and $x_2$:

$$ x_1 \; = \; a_1 \, x_2 + a_0. $$

As a result, using both as signals creates redundancy, and leads to incorrect interpretation and/or hypothesis.  This (very common) situation, is known as **collinearity** or **multicollinearity**.  A simple detection might be **Pearson's r coefficient** ($\rho$):

$$ \rho_{x1, x2} \; = \; { cov(x_1, x_2) \over \sigma_{x1} \, \sigma_{x2} }, $$

where covariance of $x_1$ and $x_2$ is divided by the product of their standard deviations.  


### Mutual Information

Avoiding using features that are linearly correlated is not enough in terms of avoiding redundant signals.  Consider the scenario below.

<img src="./mutual_info.png" width="30%" height="30%">

While the two features $x_1$ and $x_2$ do not exhibit a high decree of linear relationship, there is visiable dependency between the two variables.  For example, high $x_1$ values tend to have intermediate $x_2$ values.  A more generalized method of capturing relationship between variables is computing their **mutual information**, which measures the amount of information one would obtain for $x_1$ by knowing the corresponding $x_2$.  More precisely, mutual information $I(x_1; x_2)$ is given by

$$ I(x_1; x_2) \; = \; \sum_{x1 \in X1} \; \sum_{x2 \in X2} p(x_1, x_2) \; log \left( { p(x_1, x_2) \over p(x_1) \; p(x_2) } \right).$$

To gain some intuition about the definition above, consider two special cases.  First, if $x_1$ is completely independent of $x_2$, then joint probability $p(x_1, x_2)$ is the product of the two probabilities $p(x_1)$ and $p(x_2)$, so mutual information $I(x_1; x_2) = 0$.  Second, if $x_1$ and $x_2$ are the same distributions, then their mutual information is the same as the entropy of $x_1$, which is equivalent to entropy of $x_2$.




### Dimensionality Reduction

Given that signals used to make predictions can contain redundant information, the true dimensionality of a problem is often much smaller than the number of features available for modeling.  Therefore, **dimensionality reduction** is often carried out before modeling to parse out the set of meaningful signals to feed into the model.  There are many profound methods that have been developed for dimensionality reduction; as an example, we will only discuss a common method known as principal component analysis (**PCA**).

The geometrical intuition of PCA is straightforward.  If two features are linearly correlated (see figure above), then the data points form an ellipsoid, where the long axis is in the direction of the largest variance.  This axis becomes the first principle component, as most variance, or information, or dynamic range, happens along this dimension.  Subsequently, other principle components are identified according to their respective amount of variance.  In mathematical expressions, the principal components are the eigenvectors of the covariance matrix.       



## 2. Basic concepts for TensorFlow

You might recall from linear algebra that arrays have the following names depending on their dimensionality:

* Scalar - an array in 0-D
* Vector - an array in 1-D
* Matrix - an array in 2-D

They are all **tensors** of n-th order.  That is, a tensor is an array with arbitrary dimensionality.  Just like how scalars, vectors, and matrices can undergo mathematical operations such as addition and multiplication, tensors can be transformed with operations as well, provided that the input tensors have compatible shapes (for example, you can only add vectors with same number of elements, and multiply an NxM matrix with one that's MxP).  

TensorFlow provides a library of algorithms to perform tensor operations efficiently, which are fundemental calculation tasks in machine learning.  For example, consider a simple linear regression model with a single feature:     

$$ w_0 + w_1 x \;=\; \hat{y} $$

where standard notations are used: $x$ is the feature, $w_0$ and $w_1$ are the **weights** (or parameters) determined during model training, and $\hat{y}$ is the predicted outcome, to be compared with actual observations $y$.  The goal of building a model is to find values of $w_0$ and $w_1$ that minimize prediction error (but also balanced by the model being generalizable, i.e., not over-fit).  

### Graph representation of ML models

We can represent linear regression as a **graph**, where data flow from one node to another to undergo mathematical operations.  In this representation, we have

<img src="./linear_reg_graph.png" width="20%" height="20%">


Consider a slightly larger neural net graph:

<img src="./nnet_graph.png" width="30%" height="30%">

In this graph, there are three features $x_1$, $x_2$, and $x_3$, which are fed into **activation functions** $a_1^{(2)}$,  $a_2^{(2)}$, and $a_3^{(2)}$ in the second layer (the superscript denotes layer number and the subscript denotes the node number in that layer; note the number of nodes in each layer does not have to be the same as the number of features).  Not shown here are the weights $w$'s similar to the case of linear regression example above.  We can have different weights for different features, as well as for different activation functions.  Therefore, for each activation function $a_i^{(2)}$ in the second layer:

$$ a_i^{(2)} \;\; = g( w_{i1} x_1 + w_{i2} x_2 + w_{i3} x_3 ),$$

where $g(...)$ simply states that the activation function $a_i^{(2)}$ takes the input $w_{i1} x_1 + w_{i2} x_2 + w_{i3} x_3$.  Sometimes a **biasing term** $w_{i0}$ is added:

$$ a_i^{(2)} \; =\;g( w_{i0} + w_{i1} x_1 + w_{i2} x_2 + w_{i3} x_3 ).$$

After the operations at the second layer, the data is subsequently sent to a third layer, with activation functions $a_i^{(3)}$.  One can add arbitray numbers of nodes and layers to the neural net model; a **deep neural net** is one with many layers, perhaps 10's or 100's.  As you build more complex models using TensorFlow, it could be helpful to visualize your graph.  **[TensorBoard](https://www.tensorflow.org/versions/r0.7/how_tos/graph_viz/index.html)** provides this visualization tool.

### Activation functions

The activation functions can be different operations.  For example, if $g(...)$ is linear, then we return to linear regression.  So in practice $g(...)$ is typically non-linear, and a popular activation function is the rectified linear unit (**ReLU**):

$$g(u)\;=\;max(0, u).$$


### Model output

After the data are pass through layers of operations, the output is generated.  The range of the output value depends on what activation function is used, but generally could be any real number $[-\infty, +\infty]$.  For categorical prediction, such as a binary classification of either 0 or 1, an additional sigmoid function can be applied to bring the final output within the range of $[0, 1]$, much like the case of logistic regression:

$$ \sigma(u) \;=\; { 1 \over 1 + e^{-u} }.$$

The empty node in the neural net graph above is meant to represent this last operation.  Often we need to make a multi-class prediction, for example, given an image, if the subject is cat, dog, human, car, etc.  In this case we need to transform the output from $[-\infty, +\infty]$ to one of the several categories, and a **softmax function** would be used:

$$ S_j(\textbf{u}) \;=\; { e^{u_j} \over \sum_{k=1}^K e^{u_k} },\;\;\;\;\;j\;\in\;[1, 2,... K].$$

Here $\textbf{u}$ is an array with $K$ elements ($u_1$, $u_2$, ... $u_K$), and the function $S$ transforms $\textbf{u} \in \mathbb{R}^K$ to $[0, 1]^K$, while also observing the normalization condition

$$ \sum_{j=1}^K S_j(\textbf{u})\;=\;1.$$

In the case of multi-class prediction, each possible state is represented typically with **one-hot encoding** (also known as **dummy encoding** or **one-of-K**).  For example, with coin flip, $K$ = 2 and the two states would be (1,0) and (0,1); for dice roll, $K$ = 6 and the six states would be (1,0,0,0,0,0), (0,1,0,0,0,0), (0,0,1,0,0,0), etc.

<!---
#### Exercise
Show that the softmax function for $K$ = 1 is equivalent to a sigmoid function.
--->




## 3. Loss functions and gradient descent

In a machine learning model, we have input features $\textbf{x}$, weights $w_i$ (number of weights depends on model), and we make predictions on outcome $y$ which is denoted as $\hat{y}$.  Good models make good predictions, that is, the difference between predicted $\hat{y}$ and the actual observations $y$ should be small (without loss of model generalizability, that is, without overfitting).   

### Linear regression example

For the case of linear regression, we have two weights: $w_0$ and $w_1$, and we want the model to figure out what are good weights by minimizing prediction error, for example, an L2 **loss function** of 

$$ L\;=\;\sum (y - \hat{y}) ^ 2,$$

which is also known as squared errors (loss function is also referred to as **cost function**).  For linear regression, we can simply replace $\hat{y}$ with $w_0 + w_1 x$ and get

$$ L\;=\; L(w_0, w_1)\;=\;\sum (y - w_0 + w_1 x) ^ 2.$$

We then want to find $w_0$ and $w_1$ that minimize $L$.  A recipe is to start with some values for $w_0$ and $w_1$, change them a little bit in a way that decreases $L(w_0, w_1)$, and continue until $L(w_0, w_1)$ cannot be reduced further.  More formally, we are doing

$$ w^{new}_0 \;=\;w^{old}_0 - \alpha {\partial \over \partial w_0} L(w_0, w_1),$$
$$ w^{new}_1 \;=\;w^{old}_1 - \alpha {\partial \over \partial w_1} L(w_0, w_1).$$

The parameter $\alpha$ is the **learning rate**; large $\alpha$ means each new update of $w_0$ and $w_1$ takes a bigger leap in the direction that reduces $L(w_0, w_1)$, so it should be faster to find optimal $w_0$ and $w_1$ as less number of iterations would be needed.  However, bigger steps also mean that one could miss out on the optimal $w_0$ and $w_1$ by continuously stepping around it.  In terms of implementation, it is feasible to start with larger steps, or a faster learning rate, and then take smaller steps as the model training is approaching the miminum loss function (i.e., a **decaying learning rate**).  This iterative process of updating weights and decreasing loss function is **gradient descent**, and is a key component to machine learning algorithms.  One last terminology is **batch size**; when computing the gradient, using more than one sample helps to smooth out noises (such as due to sample variations), and the sampling size is often an adjustable parameter called batch size. 

### Another loss function

Choosing a good loss function is very important in machine learning; the squared error might not be the appropriate choice in all occasions.  For example, consider a logistic regression model where predictions $\hat{y}$ are computed via a sigmoid function

$$ \hat{y} \;=\; { 1 \over 1 + e^{ -(w_0 + w_1 x)} }.$$

The resulting $\hat{y}$ is between 0 and 1, and carries a probabilistic interpretation

$$ \hat{p}\,(y = 1) \;=\; \hat{y}, $$
$$ \hat{p}\,(y = 0) \;=\; 1 - \hat{y}.$$

Instead of comparing $\hat{y}$ with actual observations $y$, in the case of logistic regression the quality of predictions is determined by $\hat{p}\,(y = 1)$, which we want to compare with the observed probability $p\,(y = 1)$.  One way of comparing two probability distributions is **cross entropy**

$$ H(p, \hat{p}) \;=\; \sum_i \, p_i \, log(\hat{p_i}) \;=\; -y \, log(\hat{y})\,-\,(1-y)\,log(1-\hat{y}). $$

Note, cross entropy is not symmetric, so there is a distinction built into the definition between the actual $p$ and the predictive $\hat{p}$. Also, if $p$ = $\hat{p}$, we simply get back the entropy of $p$.  As $\hat{p}$ deviates from $p$, $H(p, \hat{p})$ increases, so a good error function for logistic regression is cross entropy, which one would minimize as the error function.  This is also known as the **log loss**.
